In [6]:
import numpy as np
import pandas as pd
from types import SimpleNamespace

In [7]:
par = SimpleNamespace()
par.J = 3
par.N = 10
par.K = 10000
par.F = np.arange(1,par.N+1)
par.sigma = 2
par.v = np.array([1,2,3])
par.c = 1

Question 1

In [8]:
# Set a seed
np.random.seed(80)

In [9]:
# Initialize arrays to store expected- and average realized utilities
expected_utilities = np.zeros(par.J)
avg_realized_utilities = np.zeros((par.N, par.J))

# For each career choice j
for j in range(par.J):
    # Epsilon is drawn from a normal distribution
    epsilon = np.random.normal(0, par.sigma, (par.N, par.K))
    #Calculate ex post utility
    utilities = par.v[j] + epsilon
    #Calculate expected utility
    expected_utilities[j] = par.v[j] + np.mean(epsilon)
    #Calculate average realized utilities across all simulations for each graduate 
    avg_realized_utilities[:, j] = np.mean(utilities, axis=1)


In [10]:
# Convert to DataFrame for better visualization
expected_utilities_df = pd.DataFrame(expected_utilities, columns=['Expected Utility'], index=[f'Career {j+1}' for j in range(par.J)])
avg_realized_utilities_df = pd.DataFrame(avg_realized_utilities, columns=[f'Realized Utility Career {j+1}' for j in range(par.J)], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the expected and realized utilities
print("Expected Utilities:")
print(expected_utilities_df)

print("\nRealized Utilities:")
print(avg_realized_utilities_df)

Expected Utilities:
          Expected Utility
Career 1          0.988813
Career 2          2.009004
Career 3          2.995133

Realized Utilities:
             Realized Utility Career 1  Realized Utility Career 2  \
Graduate 1                    0.997869                   2.013136   
Graduate 2                    0.968002                   2.004125   
Graduate 3                    0.996840                   2.043912   
Graduate 4                    1.002072                   2.020080   
Graduate 5                    0.983313                   2.009857   
Graduate 6                    0.973542                   1.999288   
Graduate 7                    1.014064                   1.994329   
Graduate 8                    0.978955                   1.997091   
Graduate 9                    0.973771                   2.019734   
Graduate 10                   0.999703                   1.988483   

             Realized Utility Career 3  
Graduate 1                    2.989200  
Graduate 

**Question 2**

Step 1

In [11]:
# Initialize array to store the drawn values
drawn_values = {}


# Simulate drawing J * F_i values for each person i
for i in range(par.N):
    #Determine the number of sample for the graduate
    Fi = par.F[i]
    #Generate epsilon 
    epsilon_fj = np.random.normal(0, par.sigma, (par.J, Fi, par.K))
    #Store the drawn values in the dictionary
    drawn_values[f'Graduate {i+1}'] = epsilon_fj



# Initialize array to store the prior expected utilities
prior_expected_utilities = np.zeros((par.N, par.J, par.K))


# For each graduate, i 
for i in range(par.N):
    Fi = par.F[i]
    epsilon_fj = drawn_values[f'Graduate {i+1}']
    
    # For each career track, j
    for j in range(par.J):
        #For each simulation, k
        for k in range(par.K):
            #Calculate the prior expected utilities
            prior_expected_utilities[i, j, k] = np.mean(par.v[j] + epsilon_fj[j, :, k])



# We can visualize the average prior expected utility over all simulations for simplicity
average_prior_expected_utilities = np.mean(prior_expected_utilities, axis=2)
prior_expected_utilities_df = pd.DataFrame(average_prior_expected_utilities, columns=[f'Career {j+1}' for j in range(par.J)], index=[f'Graduate {i+1}' for i in range(par.N)])



# Print the average prior expected utilities
print("Average Prior Expected Utilities:")
print(prior_expected_utilities_df)


Average Prior Expected Utilities:
             Career 1  Career 2  Career 3
Graduate 1   1.037594  1.983420  2.989729
Graduate 2   1.009278  1.984125  3.006044
Graduate 3   0.990181  2.004945  3.009577
Graduate 4   1.013475  1.981079  3.011749
Graduate 5   0.998056  2.011311  2.993387
Graduate 6   0.999810  1.986135  2.992950
Graduate 7   1.008791  2.001954  2.998760
Graduate 8   0.981806  1.997406  3.006844
Graduate 9   1.004039  2.000279  2.997815
Graduate 10  0.997350  1.998711  3.008142


Step 2

In [12]:
# Initialize array to store the chosen careers, ensuring it's 2-dimensional
chosen_careers = np.zeros((par.N, par.K), dtype=int)

# Iterates over each individual
for i in range(par.N):
    #Iterates over each career track for the given individual
    for k in range(par.K):
        # Finding the highest expected utilities
        chosen_career = np.argmax(prior_expected_utilities[i, :, k]) + 1  # Adding 1 to shift the index
        chosen_careers[i, k] = chosen_career

# Convert the chosen careers to DataFrame for visualization
chosen_careers_df = pd.DataFrame(chosen_careers, columns=[f'Simulation {k+1}' for k in range(par.K)], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print a sample of chosen careers
print("\nSample Chosen Careers:")
print(chosen_careers_df.head())


Sample Chosen Careers:
            Simulation 1  Simulation 2  Simulation 3  Simulation 4  \
Graduate 1             3             3             3             3   
Graduate 2             3             3             3             2   
Graduate 3             3             2             3             3   
Graduate 4             2             2             3             2   
Graduate 5             2             3             3             3   

            Simulation 5  Simulation 6  Simulation 7  Simulation 8  \
Graduate 1             1             3             3             2   
Graduate 2             1             3             2             2   
Graduate 3             3             3             2             3   
Graduate 4             3             2             2             3   
Graduate 5             3             3             3             3   

            Simulation 9  Simulation 10  ...  Simulation 9991  \
Graduate 1             2              3  ...                2   
Grad

Step 3

Compute the prior expectation of the value of their chosen career:

In [13]:
#Store the prior expectation of the value of their chosen career
optimal_prior_expected_utilities = np.zeros((par.N, par.K))

#Iterates over each individual
for i in range(par.N):
    #Iterates over each career track
    for k in range(par.K):
        #Assume the individual choose the optimal career track
        optimal_career = chosen_careers[i, k] - 1  # Convert to 0-index
        optimal_prior_expected_utilities[i, k] = prior_expected_utilities[i, optimal_career, k]

# Convert to DataFrame for better visualization
optimal_prior_expected_utilities_df = pd.DataFrame(optimal_prior_expected_utilities, columns=[f'Simulation {k+1}' for k in range(par.K)], index=[f'Graduate {i+1}' for i in range(par.N)])
print("\nOptimal Prior Expected Utilities:")
print(optimal_prior_expected_utilities_df.head())


Optimal Prior Expected Utilities:
            Simulation 1  Simulation 2  Simulation 3  Simulation 4  \
Graduate 1      5.592573      4.845502      5.743788      4.693154   
Graduate 2      1.692645      2.101986      3.614708      3.175421   
Graduate 3      4.575713      2.795991      3.660599      2.758500   
Graduate 4      4.908391      2.571807      4.450916      2.267958   
Graduate 5      2.541340      3.039931      3.694004      3.523570   

            Simulation 5  Simulation 6  Simulation 7  Simulation 8  \
Graduate 1      1.776984      3.250262      5.878390      5.643914   
Graduate 2      2.996473      3.357448      1.916958      3.765370   
Graduate 3      1.820960      3.605820      2.999402      3.525675   
Graduate 4      3.435880      2.005811      3.885457      3.903660   
Graduate 5      3.642964      2.543721      3.237947      1.858167   

            Simulation 9  Simulation 10  ...  Simulation 9991  \
Graduate 1      2.839372       2.301623  ...         2.713

Compute the realized value of their chosen career track

In [22]:
# Initialize the array to store the realized values
optimal_realized_values = np.zeros((par.N, par.K))

# Iterates over each individual
for i in range(par.N):
    #Iterates over each career track
    for k in range(par.K):
        # Retrieve the 1-indexed chosen career and convert to 0-index for array access
        career_index = chosen_careers[i, k] - 1
        # Compute the realized value
        noise = np.random.normal(0, par.sigma)
        optimal_realized_values[i, k] = par.v[career_index] + noise


# Convert chosen careers and realized values to DataFrame for visualization
optimal_realized_values_df = pd.DataFrame(optimal_realized_values, columns=[f'Simulation {k+1}' for k in range(par.K)], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the results
print("\nRealized Values for Chosen Careers:")
print(optimal_realized_values_df.head())


Realized Values for Chosen Careers:
            Simulation 1  Simulation 2  Simulation 3  Simulation 4  \
Graduate 1      6.197692      3.617780      2.077241      6.011708   
Graduate 2      7.597215      1.090061      4.240363      3.324734   
Graduate 3      4.142263      3.886966      4.912470      4.760200   
Graduate 4      1.035539      0.366290      2.806749      1.759414   
Graduate 5      3.518130      4.926874      3.149272      3.403586   

            Simulation 5  Simulation 6  Simulation 7  Simulation 8  \
Graduate 1     -4.688646      2.868396      4.364887      2.061822   
Graduate 2      1.257987      3.106865      4.145158      0.574873   
Graduate 3     -1.519098      3.952459      1.394390      4.790164   
Graduate 4      7.481510      1.415409     -0.701381      3.807856   
Graduate 5      3.267610      1.502662      4.246022      0.826823   

            Simulation 9  Simulation 10  ...  Simulation 9991  \
Graduate 1      0.531178       1.169558  ...         0.3

The share of graduates choosing each career

In [23]:
# Initialize the DataFrame to hold the proportions of chosen careers for each graduate
# Specify the data type as float directly at initialization, to avoid future warning.
career_proportions_df = pd.DataFrame(0.0, index=[f'Graduate {i+1}' for i in range(par.N)],
                                     columns=[f'Career {j+1}' for j in range(par.J)])


# Calculate the proportion of choices for each career for each graduate
for i in range(par.N):
    # Extract the careers chosen by each individual graduate across all simulations
    graduate_careers = chosen_careers[i, :]

    # Count occurrences of each career choice for this particular graduate
    career_counts = np.bincount(graduate_careers, minlength=par.J + 1)[1:]  # Ensures correct length and skip index 0

    # Calculate the share for each career for each graduate
    career_proportions = career_counts / par.K

    # Store the results in the DataFrame
    career_proportions_df.loc[f'Graduate {i+1}'] = career_proportions

# Print the DataFrame with the proportions of chosen careers for each graduate
print("\nProportions of Chosen Careers for Each Graduate:")
print(career_proportions_df)


Proportions of Chosen Careers for Each Graduate:
             Career 1  Career 2  Career 3
Graduate 1     0.1580    0.2991    0.5429
Graduate 2     0.0952    0.2668    0.6380
Graduate 3     0.0603    0.2527    0.6870
Graduate 4     0.0511    0.2163    0.7326
Graduate 5     0.0330    0.2101    0.7569
Graduate 6     0.0271    0.1825    0.7904
Graduate 7     0.0182    0.1711    0.8107
Graduate 8     0.0088    0.1540    0.8372
Graduate 9     0.0106    0.1449    0.8445
Graduate 10    0.0070    0.1278    0.8652


The average subjective expected utility

In [24]:
# Calculate the average subjective expected utility for each graduate
average_subjective_expected_utilities = np.mean(optimal_prior_expected_utilities, axis=1)  # Average over career tracks

# Convert to DataFrame for visualization
average_subjective_expected_utilities_df = pd.DataFrame(
    average_subjective_expected_utilities, 
    columns=['Average Subjective Expected Utility'], 
    index=[f'Graduate {i+1}' for i in range(par.N)]
)

# Print
print("Average Subjective Expected Utilities:")
print(average_subjective_expected_utilities_df)

Average Subjective Expected Utilities:
             Average Subjective Expected Utility
Graduate 1                              3.875311
Graduate 2                              3.480167
Graduate 3                              3.323341
Graduate 4                              3.234205
Graduate 5                              3.164882
Graduate 6                              3.123373
Graduate 7                              3.107815
Graduate 8                              3.089701
Graduate 9                              3.071294
Graduate 10                             3.067425


The average ex post realized utility given their choice

In [25]:
# Calculate the average realized utility for each graduate
average_realized_utilities = np.mean(realized_values, axis=1)

# Convert to DataFrame for better visualization
average_realized_utilities_df = pd.DataFrame(average_realized_utilities, columns=['Average Realized Utility'], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the results
print("\nAverage Realized Utilities for Each Graduate:")
print(average_realized_utilities_df)


Average Realized Utilities for Each Graduate:
             Average Realized Utility
Graduate 1                   2.377929
Graduate 2                   2.559496
Graduate 3                   2.607027
Graduate 4                   2.716535
Graduate 5                   2.728870
Graduate 6                   2.745551
Graduate 7                   2.808574
Graduate 8                   2.830879
Graduate 9                   2.822441
Graduate 10                  2.871346


From the above it is evident, that a larger proportion chooses Career 3, as i increases.

According to the problem, the first graduate has 1 friend in each career, the second has 2 friends, ... and the tenth has 10 friends. Which means, that the graduates with the highest number, has the most information about the expected value of each career track.

Therefore, the graduates with the highest index will observe that career 3 gives the highest utility, and a larger proportion will therefore choose career track 3.

This is also visible in the average realized utility, where the graduates with the most friends (highest index) also gets the higest average realized utility.

**Question 3**

In [26]:
#Set c = 1
par.c = 1

Step 1: Calculate the prior expected utility of each each carrer track

In [29]:
# Initialize arrays for the second year's expected utilities
second_year_expected_utilities = np.zeros((par.N, par.J, par.K))


# Iterates over each individual 
for i in range(par.N):
    Fi = par.F[i]
    epsilon_fj = drawn_values[f'Graduate {i+1}']  # Retrieve the stored epsilon values

    #Iterates over each career option
    for j in range(par.J):
        for k in range(par.K):
            #Retrieve the chosen career from period 1
            current_career = chosen_careers[i, k] - 1
            if j == current_career:
                # No switching cost if staying in the same career
                second_year_expected_utilities[i, j, k] = optimal_realized_values[i, k]
            else:
                # Apply switching cost if changing careers
                second_year_expected_utilities[i, j, k] = par.v[j] + np.mean(epsilon_fj[j, :, k]) - par.c


# Calculate average utilities over all simulations for simplicity
average_second_year_utilities = np.mean(second_year_expected_utilities, axis=2)
second_year_utilities_df = pd.DataFrame(average_second_year_utilities, columns=[f'Career {j+1}' for j in range(par.J)], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the results
print("Prior expected utility of each career track:")
print(second_year_utilities_df)

Prior expected utility of each career track:
             Career 1  Career 2  Career 3
Graduate 1  -0.179805  0.771485  1.939193
Graduate 2  -0.085987  0.926875  2.197774
Graduate 3  -0.050376  0.983404  2.382140
Graduate 4  -0.015725  0.989681  2.469675
Graduate 5  -0.022084  1.029223  2.570143
Graduate 6  -0.018390  1.011991  2.618995
Graduate 7   0.003042  1.040151  2.679588
Graduate 8  -0.019758  1.029176  2.731863
Graduate 9   0.001729  1.042907  2.744413
Graduate 10 -0.008558  1.033254  2.760205


Step 2: Each i chooses the career track with the highest expected utility

In [30]:
#Initialize array to store the chosen careers, ensuring it's 2-dimensional
chosen_careers2 = np.zeros((par.N, par.K), dtype=int)

# Determine the career track with the highest expected utility for each simulation
for i in range(par.N):
    for k in range(par.K):
        chosen_career2 = np.argmax(second_year_expected_utilities[i, :, k]) + 1  # Adding 1 to shift the index
        chosen_careers2[i, k] = chosen_career2

# Convert the chosen careers to DataFrame for visualization
chosen_careers2_df = pd.DataFrame(chosen_careers2, columns=[f'Simulation {k+1}' for k in range(par.K)], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print a sample of chosen careers
print("\nSample Chosen Careers in period 2:")
print(chosen_careers2_df.head())


Sample Chosen Careers in period 2:
            Simulation 1  Simulation 2  Simulation 3  Simulation 4  \
Graduate 1             3             3             3             3   
Graduate 2             3             3             3             2   
Graduate 3             3             2             3             3   
Graduate 4             3             3             3             2   
Graduate 5             2             3             3             3   

            Simulation 5  Simulation 6  Simulation 7  Simulation 8  \
Graduate 1             2             3             3             3   
Graduate 2             1             3             2             3   
Graduate 3             2             3             2             3   
Graduate 4             3             2             3             3   
Graduate 5             3             3             3             3   

            Simulation 9  Simulation 10  ...  Simulation 9991  \
Graduate 1             3              3  ...             

Step 3:

The prior expectation of the value of their chosen career

In [32]:
# Initialize array for the optimal second expected utilities
optimal_second_expected_utilities = np.zeros((par.N, par.K))

# Determine the optimal expected utility for each chosen career in period 2
for i in range(par.N):
    for k in range(par.K):
        optimal_career2 = chosen_careers2[i, k] - 1  # Convert to 0-index
        optimal_second_expected_utilities[i, k] = second_year_expected_utilities[i, optimal_career2, k]

# Convert to DataFrame for better visualization
optimal_second_expected_utilities_df = pd.DataFrame(optimal_second_expected_utilities, columns=[f'Simulation {k+1}' for k in range(par.K)], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the optimal second-year expected utilities
print("\nOptimal Second Year Expected Utilities:")
print(optimal_second_expected_utilities_df.head())


Optimal Second Year Expected Utilities:
            Simulation 1  Simulation 2  Simulation 3  Simulation 4  \
Graduate 1      6.197692      3.617780      2.077241      6.011708   
Graduate 2      7.597215      1.090061      4.240363      3.324734   
Graduate 3      4.142263      3.886966      4.912470      4.760200   
Graduate 4      1.998501      0.861521      2.806749      1.759414   
Graduate 5      3.518130      4.926874      3.149272      3.403586   

            Simulation 5  Simulation 6  Simulation 7  Simulation 8  \
Graduate 1      0.636985      2.868396      4.364887      3.211497   
Graduate 2      1.257987      3.106865      4.145158      1.773675   
Graduate 3     -0.397299      3.952459      1.394390      4.790164   
Graduate 4      7.481510      1.415409      1.600718      3.807856   
Graduate 5      3.267610      1.502662      4.246022      0.826823   

            Simulation 9  Simulation 10  ...  Simulation 9991  \
Graduate 1      1.103222       1.169558  ...        

Realized value of their choosen career track

In [34]:
# Initialize storage for first and second year utilities
realized_values = np.random.rand(par.N, par.K) 
second_year_realized_utilities = np.zeros((par.N, par.J, par.K))

# Iterate for each individual
for i in range(par.N):
    Fi = par.F[i]
    epsilon_fj = drawn_values[f'Graduate {i+1}']  # Retrieve the stored epsilon values

    # Iterate for each career track
    for j in range(par.J):
        for k in range(par.K):
            current_career2 = chosen_careers2[i, k] - 1
            if j == current_career2:
                # No switching cost if staying in the same career
                second_year_realized_utilities[i, j, k] = optimal_realized_values[i, k]
            else:
                # Apply switching cost if changing careers
                second_year_realized_utilities[i, j, k] = expected_utilities[j] - par.c

# Calculate average utilities over all simulations for simplicity
average_second_year_utilities = np.mean(second_year_realized_utilities, axis=2)
second_year_utilities_df = pd.DataFrame(average_second_year_utilities, columns=[f'Career {j+1}' for j in range(par.J)], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the results
print("Average Second-Year Realized Utilities Considering Switching Costs:")
print(second_year_utilities_df)

Average Second-Year Realized Utilities Considering Switching Costs:
             Career 1  Career 2  Career 3
Graduate 1   0.191386  1.338404  2.430441
Graduate 2   0.114881  1.303514  2.566138
Graduate 3   0.063314  1.275005  2.696767
Graduate 4   0.059592  1.209486  2.764089
Graduate 5   0.026454  1.209076  2.823926
Graduate 6   0.016030  1.157311  2.883800
Graduate 7   0.011123  1.147246  2.955453
Graduate 8  -0.001400  1.114057  2.993390
Graduate 9   0.000082  1.110601  2.998215
Graduate 10 -0.009044  1.067705  3.034431


The share of graduates

In [35]:
# Initialize the DataFrame to hold the proportions of chosen careers for each graduate
career_proportions2_df = pd.DataFrame(0.0, index=[f'Graduate {i+1}' for i in range(par.N)],
                                     columns=[f'Career {j+1}' for j in range(par.J)])


# Calculate the proportion of choices for each career for each graduate
for i in range(par.N):
    # Extract the careers chosen by each individual graduate across all simulations
    graduate_careers2 = chosen_careers2[i, :]

    # Count occurrences of each career choice for this particular graduate
    career_counts2 = np.bincount(graduate_careers2, minlength=par.J + 1)[1:]  # Ensure correct length and skip index 0

    # Calculate the share for each career for this graduate
    career_proportions2 = career_counts2 / par.K

    # Store the results in the DataFrame
    career_proportions2_df.loc[f'Graduate {i+1}'] = career_proportions2

# Print the DataFrame with the proportions of chosen careers for each graduate
print("\nProportions of Chosen Careers for Each Graduate period 2:")
print(career_proportions2_df)


Proportions of Chosen Careers for Each Graduate period 2:
             Career 1  Career 2  Career 3
Graduate 1     0.1404    0.2893    0.5703
Graduate 2     0.0987    0.2731    0.6282
Graduate 3     0.0735    0.2596    0.6669
Graduate 4     0.0618    0.2432    0.6950
Graduate 5     0.0452    0.2348    0.7200
Graduate 6     0.0377    0.2298    0.7325
Graduate 7     0.0366    0.2246    0.7388
Graduate 8     0.0263    0.2135    0.7602
Graduate 9     0.0240    0.2076    0.7684
Graduate 10    0.0211    0.2077    0.7712


The average subjective expected utility

In [37]:
# Calculate the overall average expected utility for each graduate
second_subjective = np.mean(optimal_second_expected_utilities, axis=(1))  # Average over both careers and simulations

# Convert to DataFrame for visualization
second_subjective_df = pd.DataFrame(second_subjective, columns=['Overall Average Expected Utility'], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the overall average expected utilities
print("Second period average subjective utility:")
print(second_subjective_df)

Second period average subjective utility:
             Overall Average Expected Utility
Graduate 1                           2.875205
Graduate 2                           2.885375
Graduate 3                           2.951504
Graduate 4                           2.967409
Graduate 5                           3.020812
Graduate 6                           3.011450
Graduate 7                           3.062978
Graduate 8                           3.074691
Graduate 9                           3.086411
Graduate 10                          3.076695


Ex post realized utility given their choice

In [45]:
# Calculate the average realized utility for each graduate
average_realized_utilities2 = np.mean(second_year_realized_utilities, axis=(1, 2))

# Convert to DataFrame for better visualization
average_realized_utilities2_df = pd.DataFrame(average_realized_utilities2, columns=['Average Realized Utility'], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the results
print("\nAverage Realized Utilities for Each Graduate:")
print(average_realized_utilities2_df)



Average Realized Utilities for Each Graduate:
             Average Realized Utility
Graduate 1                   1.320077
Graduate 2                   1.328178
Graduate 3                   1.345029
Graduate 4                   1.344389
Graduate 5                   1.353152
Graduate 6                   1.352380
Graduate 7                   1.371274
Graduate 8                   1.368682
Graduate 9                   1.369633
Graduate 10                  1.364364


The share of graduates that choose to switch careers conditional on which career they chose in the first year

In [48]:
# Initialize an array to store the count of switches for each graduate
switch_count_individual = np.zeros(par.N)

# Iterate over each individual and each simulation
for i in range(par.N):
    switch_count = 0
    for k in range(par.K):
        initial_career = chosen_careers[i, k] - 1  # Convert to 0-index
        second_career = chosen_careers2[i, k] - 1  # Convert to 0-index

        if initial_career != second_career:
            switch_count += 1  # Increment switch count if careers are different

    # Calculate the percentage of switches for this graduate
    switch_count_individual[i] = (switch_count / par.K) * 100

# Convert to DataFrame for better visualization
switch_count_individual_df = pd.DataFrame(switch_count_individual, columns=['Percentage of Career Switches'], index=[f'Graduate {i+1}' for i in range(par.N)])

# Print the results
print("\nPercentage of Career Switches for Each Graduate:")
print(switch_count_individual_df)


Percentage of Career Switches for Each Graduate:
             Percentage of Career Switches
Graduate 1                           29.53
Graduate 2                           26.11
Graduate 3                           23.45
Graduate 4                           22.31
Graduate 5                           21.28
Graduate 6                           20.75
Graduate 7                           19.78
Graduate 8                           19.39
Graduate 9                           18.77
Graduate 10                          19.32


From the analysis, it is clear that there is a trend between having fewer friends and a higher percentage of career switches. Individuals with fewer friends, and consequently less information in the first period, are more likely to switch careers.

This trend is logical because with less information initially, these individuals are more likely to choose a career with a lower payoff. By the second period, they have gained more information about which careers offer higher payoffs, making them more likely to switch to a better option.